In [1]:
# Suppressing Warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Importing Pandas and NumPy
import pandas as pd, numpy as np

In [3]:
# Importing all datasets
ecom_data = pd.read_csv("ecommerce_consumers.csv")
ecom_data.head()

,ratio,time,label
0,0.54,17.2,female
1,0.93,18.2,male
2,0.84,13.6,female
3,0.19,6.0,male
4,0.89,13.2,female


In [4]:
ecom_data.label.replace({"female":0, "male":1},inplace = True)

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
X = ecom_data.drop(['label'], axis=1)
y = ecom_data['label']

In [7]:
# Splitting the data into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, test_size=0.3, random_state=100)

In [8]:
from sklearn.preprocessing import MinMaxScaler

In [9]:
scaler = MinMaxScaler()

X_train[['time']] = scaler.fit_transform(X_train[['time']])

X_train.head()

,ratio,time
74,0.35,0.466667
3,0.19,0.000000
185,0.38,0.777778
26,0.02,0.288889
90,0.60,0.422222


In [10]:
type(y_train)

pandas.core.series.Series

In [11]:
y_train.info()

<class 'pandas.core.series.Series'>
Int64Index: 140 entries, 74 to 8
Series name: label
Non-Null Count  Dtype
--------------  -----
140 non-null    int64
dtypes: int64(1)
memory usage: 2.2 KB


In [12]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 140 entries, 74 to 8
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ratio   140 non-null    float64
 1   time    140 non-null    float64
dtypes: float64(2)
memory usage: 3.3 KB


In [13]:
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()

X_train_sm = sm.add_constant(X_train)
logm2 = sm.GLM(y_train,X_train_sm, family = sm.families.Binomial())
res = logm2.fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                  label   No. Observations:                  140
Model:                            GLM   Df Residuals:                      137
Model Family:                Binomial   Df Model:                            2
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -89.320
Date:                Wed, 09 Aug 2023   Deviance:                       178.64
Time:                        07:55:33   Pearson chi2:                     140.
No. Iterations:                     4   Pseudo R-squ. (CS):          0.0002975
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.6969      0.470      1.484      0.138      -0.223       1.617
ratio          0.0674      0.607      0.111      0.912      -1.122       1.256
time          -0.1017      0.609     -0.167      0.867      -1.296       1.092
==============================================================================
"""

In [14]:
# Getting the predicted values on the train set
y_train_pred = res.predict(X_train_sm)
y_train_pred[:10]

74     0.662174
3      0.670328
185    0.655518
26     0.661241
90     0.666939
127    0.662590
81     0.661893
88     0.655661
119    0.660211
110    0.650580
dtype: float64

In [15]:
y_train_pred_final = pd.DataFrame({'Sex':y_train.values, 'Sex_Prob':y_train_pred})
#y_train_pred_final['CustID'] = y_train.index
y_train_pred_final.head()

,Sex,Sex_Prob
74,0,0.662174
3,1,0.670328
185,1,0.655518
26,1,0.661241
90,0,0.666939


In [16]:
y_train_pred['predicted'] = y_train_pred.map(lambda x: 1 if x > 0.5 else 0)

# Let's see the head
y_train_pred_final.head()

,Sex,Sex_Prob
74,0,0.662174
3,1,0.670328
185,1,0.655518
26,1,0.661241
90,0,0.666939


In [17]:
from sklearn import metrics

In [18]:
# Confusion matrix 
confusion = metrics.confusion_matrix(y_train_pred_final.Sex, y_train_pred_final.predicted )
print(confusion)

AttributeError: 'DataFrame' object has no attribute 'Churn'

In [19]:
# Let's check the overall accuracy.
print(metrics.accuracy_score(y_train_pred_final.Churn, y_train_pred_final.predicted))

AttributeError: 'DataFrame' object has no attribute 'predicted'